In [ ]:
import torch
import MySQLdb
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torch.utils.data.sampler as smp
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import IPython.display
from torch.autograd import Variable
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from PIL import Image

import dataset
import dataLoader
import configure as cf
import plot_utils as utils
import train_function as train

In [ ]:
#define transform function, define trainset and valset
#AlexNet requires the input size of 224*224*3
imgTransform = transforms.Compose([transforms.Scale((224)),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor()                       
                            ])
trainLoader, valLoader = dataLoader.get_train_valid_loader(cf.photo_url,50,32,'food',imgTransform,0.1,-1)

In [ ]:
# add more linear layers and another more dropout layer
# the loss utimately is 0.445
# adjust learning rate from 1e-2 to 1e-3
# the network still overfit on 2rd epoch
class AlexNet(nn.Module):

    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, 512),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x

In [ ]:
#define learningRate
learningRate = 1e-3

# Definition of our network.
# network = models.alexnet(pretrained = True, num_classes=1)
network = AlexNet()

#Definition of our loss.
#The MSELoss function (input? output?)
criterion = nn.MSELoss()

# Definition of optimization strategy.
optimizer = optim.SGD(network.parameters(), lr = learningRate)

result = []
# Train the previously defined model.
result = train.train_model(network, criterion, optimizer, trainLoader, valLoader,
                           n_epochs = 10, use_gpu = True)

In [ ]:
# show plot 
# the result shoule be multipled by the batchsize
trainLoss = result[2]
for i in range (0, len(trainLoss)):
    trainLoss[i] *= 50
valLoss = result[3]
for i in range (0, len(valLoss)):
    valLoss[i] *= 50
# record the result
print("trainLoss data: ")
print(trainLoss)
print("valLoss data: ")
print(valLoss)
utils.plot_loss(trainLoss,valLoss)
utils.plot_accuracy(result[0],result[1])